<a href="https://colab.research.google.com/github/abdyraman/quantium/blob/main/quantium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
behaviors_df=pd.read_csv('QVI_purchase_behaviour.csv')
transactions_df=pd.read_excel('QVI_transaction_data.xlsx')


In [5]:
behaviors_df.sample(10)

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
15299,56153,MIDAGE SINGLES/COUPLES,Premium
46028,172156,YOUNG FAMILIES,Budget
60168,227093,OLDER FAMILIES,Mainstream
21307,77495,MIDAGE SINGLES/COUPLES,Mainstream
59492,225049,RETIREES,Mainstream
57780,217308,YOUNG FAMILIES,Mainstream
19143,70061,RETIREES,Premium
59261,223151,YOUNG FAMILIES,Budget
63623,238320,OLDER FAMILIES,Premium
56884,215088,YOUNG SINGLES/COUPLES,Mainstream


In [6]:
transactions_df.sample(10)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
26406,43534,111,111302,113481,52,Grain Waves Sour Cream&Chives 210G,1,3.6
253829,43471,210,210100,209338,47,Doritos Corn Chips Original 170g,2,8.8
175316,43460,67,67109,64463,107,Smiths Crinkle Cut French OnionDip 150g,2,5.2
19504,43426,78,78192,76573,57,Old El Paso Salsa Dip Tomato Mild 300g,2,10.2
176996,43450,100,100088,98901,17,Kettle Sensations BBQ&Maple 150g,2,9.2
90776,43499,144,144069,144262,70,Tyrrells Crisps Lightly Salted 165g,2,8.4
96014,43457,221,221280,221107,62,Pringles Mystery Flavour 134g,2,7.4
72034,43425,59,59119,55103,87,Infuzions BBQ Rib Prawn Crackers 110g,2,7.6
36382,43377,48,48141,43916,3,Kettle Sensations Camembert & Fig 150g,2,9.2
208260,43415,91,91371,90536,71,Twisties Cheese Burger 250g,2,8.6
